# Code for competition

Libraries imports and initializations

In [21]:
import os
import os.path
import random

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from PIL import Image
from sklearn.metrics import (accuracy_score, confusion_matrix, f1_score,
                             precision_score, recall_score)
from sklearn.utils import class_weight, shuffle

tfk = tf.keras
tfkl = tf.keras.layers
print(f"Tensorflow version = ", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

Tensorflow version =  2.10.0
Num GPUs Available:  1


Different directories whether the code is executed on Kaggle or on a local machine

In [22]:
kaggle_dir = "/kaggle/input/competition"
local_dir = os.getcwd() + '/training_data_final'


Load dataset and automatically splitting in training and validation datasets

In [23]:
input_shape = (96, 96, 3)
input_size = input_shape[:-1]
batch_gen = 128
dir = local_dir

traininig_set = tfk.utils.image_dataset_from_directory(
    directory=dir,
    validation_split=0.2,
    subset="training",
    seed=seed,
    image_size=(96, 96),
    batch_size=batch_gen)

validation_set = tfk.utils.image_dataset_from_directory(
    directory=dir,
    validation_split=0.2,
    subset="validation",
    seed=seed,
    image_size=(96, 96),
    batch_size=batch_gen)

labels = traininig_set.class_names
print("labels = ", labels)


Found 3542 files belonging to 8 classes.
Using 2834 files for training.
Found 3542 files belonging to 8 classes.
Using 708 files for validation.
labels =  ['Species1', 'Species2', 'Species3', 'Species4', 'Species5', 'Species6', 'Species7', 'Species8']


Augmentation of the dataset in an online fashion as pre-processing layers to be inserted in the model

In [24]:
normalization_layer = tfkl.Rescaling(1.0 / 255)

translation_layer = tfkl.RandomTranslation(
    height_factor=0.3,
    width_factor=0.3,
    fill_mode='reflect',
    seed=seed,
)

rotation_layer = tfkl.RandomRotation(
    factor=0.3,
    fill_mode='reflect',
    seed=seed,
)

zoom_layer = tfkl.RandomZoom(
    height_factor=0.3,
    width_factor=0.3,
    fill_mode='reflect',
    seed=seed,
)

contrast_layer = tfkl.RandomContrast(factor=0.1, seed=seed)

flip_layer = tfkl.RandomFlip(mode="horizontal", seed=seed)

brightness_layer = tfkl.RandomBrightness(factor=0.2, seed=seed)


# Note: Data augmentation is inactive at test time so input images will only be augmented during calls to Model.fit
# (not Model.evaluate or Model.predict).
data_augmentation_preprocessing_layers = tfk.Sequential(
    [normalization_layer, translation_layer, rotation_layer, zoom_layer, contrast_layer, brightness_layer, flip_layer])


#In alternative: https://www.tensorflow.org/tutorials/images/data_augmentation#random_transformations

Some tests to do with the augmentation, to see results, and check what works better

In [25]:
#layers.adapt method applies the modifications, useful to show the results of the augmentation
# https://www.tensorflow.org/guide/keras/preprocessing_layers#the_adapt_method
#layer.adapt(image)


# applies data augmentation to the training set before training
#train_dataset = train_dataset.batch(16).map(lambda x, y: (data_augmentation(x), y))


NameError: name 'layer' is not defined

Compute class weights automatically (or not...)

In [26]:
# Compute the class weights in order to balance loss during training
# TODO: I'll fix this later
'''
y_numeric = []
for v in y_val:
    y_numeric.append(np.argmax(v))


class_weights = dict(enumerate(class_weight.compute_class_weight(
    'balanced', classes=labels, y=y_numeric)))
print(class_weights)


print(f"training set input shape", traininig_set.shape,)
print(f"validation set input shape", validation_set.shape)

'''


'\ny_numeric = []\nfor v in y_val:\n    y_numeric.append(np.argmax(v))\n\n\nclass_weights = dict(enumerate(class_weight.compute_class_weight(\n    \'balanced\', classes=labels, y=y_numeric)))\nprint(class_weights)\n\n\nprint(f"training set input shape", traininig_set.shape,)\nprint(f"validation set input shape", validation_set.shape)\n\n'

Models metadata

In [27]:

labels = {0: "Species1", 1: "Species2", 2: "Species3", 3: "Species4",
          4: "Species5", 5: "Species6", 6: "Species7", 7: "Species8"}


Online augmentation with dataset mapping requires caching and prefetching in order to avoid bottlenecking the CPU

In [ ]:

#train_ds = traininig_set.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
#val_ds = validation_set.cache().prefetch(buffer_size=tf.data.AUTOTUNE)


### Models definition

In [30]:
def build_stupid_model(input_shape):
	tf.random.set_seed(seed)

	# stupid model just to test this shit

	input_layer = tfk.Input(shape=input_shape)
	x = data_augmentation_preprocessing_layers(input_layer)

	convolution = tfk.Sequential([
		tfkl.Conv2D(128, 3, padding='same', activation='relu'),
		tfkl.MaxPooling2D(),
		tfkl.Conv2D(128, 3, padding='same', activation='relu'),
		tfkl.MaxPooling2D(),
		tfkl.Conv2D(128, 3, padding='same', activation='relu'),
		tfkl.MaxPooling2D(),
		tfkl.Flatten(),
		tfkl.Dense(128, activation='relu'),
	])

	x = convolution(x)
	output_layer = tfkl.Dense(
		units=len(labels),
		activation='softmax',
		kernel_initializer=tfk.initializers.GlorotUniform(seed),
		name='output_layer')(x)

	# Connect input and output through the Model class
	model = tfk.Model(inputs = input_layer, outputs = output_layer, name = 'stupidity')

	def get_lr_metric(optimizer):
		def lr(y_true, y_pred):
			return optimizer._decayed_lr(tf.float32) # I use ._decayed_lr method instead of .lr method because the later one is not working for me
		return lr
	optimizer = tfk.optimizers.Adam()
	lr_metric = get_lr_metric(optimizer)

	# Compile the model
	model.compile(loss=tfk.losses.SparseCategoricalCrossentropy(), optimizer=optimizer, metrics=['accuracy', lr_metric])

	return model


In [31]:
tf.get_logger().setLevel('ERROR') # stupid warnings that can be discarded

model = build_stupid_model(input_shape)

model.summary()


Model: "stupidity"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 sequential_4 (Sequential)   (None, 96, 96, 3)         0         
                                                                 
 sequential_6 (Sequential)   (None, 128)               2658176   
                                                                 
 output_layer (Dense)        (None, 8)                 1032      
                                                                 
Total params: 2,659,208
Trainable params: 2,659,208
Non-trainable params: 0
_________________________________________________________________


## Model Training

In [32]:
# good GPU utilization on my machine with this batch size
batch_size = 64
epochs = 400

# exponential decay for the learning rate
learn_strating_rate = 1e-3 # suggested 5e-5 for transfer learning applications
def scheduler(epoch, lr):
    return learn_strating_rate * tf.math.exp(- epoch / 50.0)

early_stop = tfk.callbacks.EarlyStopping(
    monitor='val_accuracy', mode='max', patience=30, restore_best_weights=True)

learning_rate_scheduler = tfk.callbacks.LearningRateScheduler(scheduler)

# training
history = model.fit(
    x=traininig_set,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=validation_set,
    
    # class_weight=class_weights, # TODO: I have to sleep
    callbacks=[early_stop, learning_rate_scheduler]
).history


Epoch 1/400
 5/23 [=====>........................] - ETA: 6:37 - loss: 6.3138 - accuracy: 0.1297 - lr: 0.0010

### Plot training results

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(history['loss'], label='Std training',
         alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(history['val_loss'], label='Std validation',
         alpha=.8, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(10, 5))
plt.plot(history['accuracy'], label='Std training',
         alpha=.8, color='#ff7f0e', linestyle='--')
plt.plot(history['val_accuracy'], label='Std validation',
         alpha=.8, color='#ff7f0e')
plt.legend(loc='upper right')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()


Plot the confusion matrix (evaluated on the validation set)

In [ ]:
predictions = model.predict(X_val)
cm = confusion_matrix(np.argmax(y_val, axis=-1),
                      np.argmax(predictions, axis=-1))

accuracy = accuracy_score(np.argmax(y_val, axis=-1),
                          np.argmax(predictions, axis=-1))
precision = precision_score(
    np.argmax(y_val, axis=-1), np.argmax(predictions, axis=-1), average='macro')
recall = recall_score(np.argmax(y_val, axis=-1),
                      np.argmax(predictions, axis=-1), average='macro')
f1 = f1_score(np.argmax(y_val, axis=-1),
              np.argmax(predictions, axis=-1), average='macro')
print('Accuracy:', accuracy.round(4))
print('Precision:', precision.round(4))
print('Recall:', recall.round(4))
print('F1:', f1.round(4))

plt.figure(figsize=(6, 5.5))
sns.heatmap(cm.T, xticklabels=labels, yticklabels=labels)
plt.ylabel('True labels')
plt.xlabel('Predicted labels')
plt.show()


Plot one example of an image for each class from the validation set of images.
For each image show the prediction on a bar plot

In [ ]:
fig, axes = plt.subplots(8, 2)
fig.set_size_inches(15, 30)

example_prediction = [0] * 8

for i in range(8):
    example_from_validation = -1

    while example_from_validation == -1:
        example_from_validation = random.choice(range(len(X_val)))
        if np.argmax(y_val[example_from_validation]) == i:
            example_prediction[i] = example_from_validation
        else:
            example_from_validation = -1

    predicted = model.predict(
        X_val[example_prediction[i]].reshape(1, 96, 96, 3))

    axes[i, 0].imshow(X_val[example_prediction[i]])
    axes[i, 0].set_title(
        'True label: ' + labels[np.argmax(y_val[example_prediction[i]])])
    axes[i, 1].barh(list(labels.values()), predicted[0],
                    color=plt.get_cmap('Paired').colors)
    axes[i, 1].set_title('Predicted label: ' + labels[np.argmax(predicted)])
    axes[i, 1].grid(alpha=.3)


plt.show()


### Save the model

Here it is not working

In [ ]:
restored_model = tfk.models.load_model('simo_model')

# TODO: not right because validation set can change
restored_loss, restored_acc = restored_model.evaluate(X_val, y_val, verbose=2)
loss, acc = model.evaluate(X_val, y_val, verbose=2)
if acc > restored_acc:  # know that this is conceptually wrong
    print("Model improved!")
    model.save('simo_model')
else:
    print("No improvement!")


17/17 - 2s - loss: 0.2023 - accuracy: 0.9229
17/17 - 1s - loss: 1.0335 - accuracy: 0.6654
No improvement!
